In [1]:
# importamos librerias que vamos a necesitar
import pandas as pd 
import gzip
import ast

con el archivo descomprimido desde el escritorio, quedamos a disposicion de su archivo json en el cual estaremos realizando el proceso de ETL

In [2]:
#lista vacia donde ingresaremos los datos del archivo
items = []
#abrimos archivo json
with open('dataset_original/australian_users_items.json', 'r', encoding='utf-8') as f:
    #creamos un bucle for para iterar e ingresar los datos dentro de la lista vacia
    for line in f.readlines():
        items.append(ast.literal_eval(line))
#creamos dataframe con los datos
user_items = pd.DataFrame(items)
#abrimos dataframe
user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


chequeo de valores nulos

In [3]:
# verificamos valores nulos
user_items.isnull().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

In [4]:
# eliminamos columna 'user_url' ya que no sera relevante para el proyecto
user_items= user_items.drop(columns=['user_url'], axis=1)

In [5]:
# data de la columna 'items' que vamos a desanidar
user_items['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

# transformacion

desanidamos los datos de la columna items

In [6]:
# desanidamos la columna 'items'
user_items_2 = pd.json_normalize(user_items['items'].explode())

In [7]:
# Ahora unimos el DataFrame expandido con el original
user_items= user_items_2.join(user_items)

In [8]:
user_items

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,items
0,10,Counter-Strike,6.0,0.0,76561197970982479,277.0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,20,Team Fortress Classic,0.0,0.0,js41637,888.0,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,30,Day of Defeat,7.0,0.0,evcentric,137.0,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,40,Deathmatch Classic,0.0,0.0,Riot-Punch,328.0,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,50,Half-Life: Opposing Force,0.0,0.0,doctr,541.0,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...,...,...,...
5170010,373330,All Is Dust,0.0,0.0,NaN,NaN,NaN,NaN
5170011,388490,One Way To Die: Steam Edition,3.0,3.0,NaN,NaN,NaN,NaN
5170012,521570,You Have 10 Seconds 2,4.0,4.0,NaN,NaN,NaN,NaN
5170013,519140,Minds Eyes,3.0,3.0,NaN,NaN,NaN,NaN


In [9]:
# eliminamos la columna 'items' ya que no la necesitamos debido a que desanidamos
user_items = user_items.drop('items', axis=1)

In [10]:
# eliminamos las filas donde todas sus columnas sean vacias
user_items= user_items.dropna(how='all')

chequeo de valores nulos

In [11]:
# verificamos valores nulos
user_items.isnull().sum()

item_id                 134
item_name               134
playtime_forever        134
playtime_2weeks         134
user_id             5065033
items_count         5065033
steam_id            5065033
dtype: int64

In [12]:
# eliminamos columnas que no necesitaremos
user_items=user_items.drop(columns=['playtime_2weeks', 'steam_id'])

In [13]:
user_items.head()

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6.0,76561197970982479,277.0
1,20,Team Fortress Classic,0.0,js41637,888.0
2,30,Day of Defeat,7.0,evcentric,137.0
3,40,Deathmatch Classic,0.0,Riot-Punch,328.0
4,50,Half-Life: Opposing Force,0.0,doctr,541.0


In [14]:
# aqui vemos la cantidad de usuarios que no tienen ningun juego registrado
usuarios_no_juegos= user_items[user_items['items_count']==0].shape[0]
print(f'usuarios sin ningun juego registrado: {usuarios_no_juegos}')

usuarios sin ningun juego registrado: 16806


In [15]:
# eliminamos los usuarios que no tengan juegos (items_count) ya que no seran util en este proyecto
user_items= user_items[user_items['items_count']>0]

In [16]:
user_items

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6.0,76561197970982479,277.0
1,20,Team Fortress Classic,0.0,js41637,888.0
2,30,Day of Defeat,7.0,evcentric,137.0
3,40,Deathmatch Classic,0.0,Riot-Punch,328.0
4,50,Half-Life: Opposing Force,0.0,doctr,541.0
...,...,...,...,...,...
88303,213650,Dwarfs F2P,62.0,76561198320136420,321.0
88304,219340,The Banner Saga: Factions,11.0,ArkPlays7,4.0
88305,202090,Magicka: Wizard Wars,1501.0,76561198323066619,22.0
88306,239220,The Mighty Quest For Epic Loot,2374.0,76561198326700687,177.0


In [17]:
# tambien podemos observar que una gran cantidad de usuarios que no tienen horas de juego registradas
no_horas_juego= user_items[user_items['playtime_forever']==0].shape[0]
print(f'cantidad de usuarios sin horas de juegos registradas: {no_horas_juego}')

cantidad de usuarios sin horas de juegos registradas: 23064


In [18]:
#eliminamos todos los usuarios que no tengan ninguna hora de juego registrada
user_items= user_items[user_items['playtime_forever']>0]

In [19]:
user_items

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6.0,76561197970982479,277.0
2,30,Day of Defeat,7.0,evcentric,137.0
8,300,Day of Defeat: Source,4733.0,maplemage,629.0
10,3830,Psychonauts,333.0,corrupted_soul,115.0
12,3900,Sid Meier's Civilization IV,338.0,jorellpogi,71.0
...,...,...,...,...,...
88303,213650,Dwarfs F2P,62.0,76561198320136420,321.0
88304,219340,The Banner Saga: Factions,11.0,ArkPlays7,4.0
88305,202090,Magicka: Wizard Wars,1501.0,76561198323066619,22.0
88306,239220,The Mighty Quest For Epic Loot,2374.0,76561198326700687,177.0


In [20]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48328 entries, 0 to 88308
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_id           48328 non-null  object 
 1   item_name         48328 non-null  object 
 2   playtime_forever  48328 non-null  float64
 3   user_id           48328 non-null  object 
 4   items_count       48328 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.2+ MB


In [21]:
# cambiamos tipo de dato de la columna 'items_count'
user_items['items_count']=user_items['items_count'].astype(int)

C:\Users\bryam\AppData\Local\Temp\ipykernel_17216\4135068061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items['items_count']=user_items['items_count'].astype(int)


In [22]:
# cambiamos tipo de dato de la columna 'item_id'
user_items['item_id']=user_items['item_id'].astype(int)

C:\Users\bryam\AppData\Local\Temp\ipykernel_17216\4258556098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_items['item_id']=user_items['item_id'].astype(int)


In [23]:
user_items=user_items.reset_index(drop=True)

In [24]:
user_items

,item_id,item_name,playtime_forever,user_id,items_count
0,10,Counter-Strike,6.0,76561197970982479,277
1,30,Day of Defeat,7.0,evcentric,137
2,300,Day of Defeat: Source,4733.0,maplemage,629
3,3830,Psychonauts,333.0,corrupted_soul,115
4,3900,Sid Meier's Civilization IV,338.0,jorellpogi,71
...,...,...,...,...,...
48323,213650,Dwarfs F2P,62.0,76561198320136420,321
48324,219340,The Banner Saga: Factions,11.0,ArkPlays7,4
48325,202090,Magicka: Wizard Wars,1501.0,76561198323066619,22
48326,239220,The Mighty Quest For Epic Loot,2374.0,76561198326700687,177


In [25]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48328 entries, 0 to 48327
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   item_id           48328 non-null  int32  
 1   item_name         48328 non-null  object 
 2   playtime_forever  48328 non-null  float64
 3   user_id           48328 non-null  object 
 4   items_count       48328 non-null  int32  
dtypes: float64(1), int32(2), object(2)
memory usage: 1.5+ MB


# guardamos archivo a clean_user_items.parquet

In [27]:
ruta='data/clean_user_items.parquet'
user_items.to_parquet(ruta, index=False)